In [5]:
import json
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# 1. 파생변수 생성 함수 (학습 시와 동일하게 유지)
def add_catboost_features(df):
    X = df.copy()
    X['HasBalance'] = (X['Balance'] > 0).astype(int)
    X['BalanceSalaryRatio'] = X['Balance'] / (X['EstimatedSalary'] + 1e-6)
    X['Age_Group'] = pd.cut(X['Age'], bins=[0, 30, 45, 60, 100], labels=[0, 1, 2, 3]).astype(int)
    X['Prod_is_1'] = (X['NumOfProducts'] == 1).astype(int)
    X['Prod_is_2'] = (X['NumOfProducts'] == 2).astype(int)
    X['Prod_ge_3'] = (X['NumOfProducts'] >= 3).astype(int)
    X['ZeroBal_Prod1'] = ((X['Balance'] == 0) & (X['NumOfProducts'] == 1)).astype(int)
    X['ZeroBal_Prod2'] = ((X['Balance'] == 0) & (X['NumOfProducts'] == 2)).astype(int)
    X['Prod2_Inactive'] = ((X['NumOfProducts'] == 2) & (X['IsActiveMember'] == 0)).astype(int)
    X['Inactive_Old'] = ((X['IsActiveMember'] == 0) & (X['Age'] >= 45)).astype(int)
    return X

# 2. 데이터 준비
df = pd.read_csv('../data/Customer-Churn-Records.csv') # 경로 확인 필요
X_enriched = add_catboost_features(df)
X = X_enriched.drop(['RowNumber', 'CustomerId', 'Surname', 'Complain', 'Exited'], axis=1)
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. 모델 학습
cat_features = ['Geography', 'Gender', 'Card Type']
model = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    scale_pos_weight=4,
    cat_features=cat_features,
    random_state=42,
    verbose=100
)
model.fit(X_train, y_train)

# --- [핵심] 4. 모델 및 메타데이터 저장 ---

# 4-1) 모델 저장 (경로 에러 방지를 위해 현재 폴더에 영어 이름으로 저장)
# UnicodeDecodeError 방지를 위해 단순한 이름을 사용합니다.
model_filename = "catboost_churn.cbm"
model.save_model(model_filename)

# 4-2) Streamlit 입력을 위한 메타데이터 구성
meta = {
    "feature_names": list(X.columns),
    "cat_features": cat_features,
    "threshold": 0.4,
    "num_stats": {},
    "cat_values": {}
}

for col in X.columns:
    if col in cat_features:
        # 범주형: 고유값 리스트 저장 (Streamlit Selectbox용)
        meta["cat_values"][col] = sorted(X_train[col].dropna().astype(str).unique().tolist())
    else:
        # 수치형: 통계값 저장 (Streamlit Number Input 범위용)
        s = pd.to_numeric(X_train[col], errors="coerce")
        meta["num_stats"][col] = {
            "min": float(np.nanmin(s)),
            "max": float(np.nanmax(s)),
            "median": float(np.nanmedian(s))
        }

# 4-3) JSON 저장
with open("model_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료!")
print(f"- 모델 파일: {model_filename}")
print(f"- 메타데이터: model_meta.json")

0:	learn: 0.6702011	total: 47.3ms	remaining: 23.6s
100:	learn: 0.4331121	total: 3.83s	remaining: 15.1s
200:	learn: 0.4006598	total: 7.32s	remaining: 10.9s
300:	learn: 0.3591268	total: 11.1s	remaining: 7.37s
400:	learn: 0.3243952	total: 15.1s	remaining: 3.72s
499:	learn: 0.2978853	total: 18.8s	remaining: 0us
✅ 저장 완료!
- 모델 파일: catboost_churn.cbm
- 메타데이터: model_meta.json
